Проверка работоспосбоности nltk, path- указка пути скачивания

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

print(nltk.data.path)

nltk.data.path.append('C:/Users/Slyexistence/nltk_data')
nltk.download('punkt', download_dir='C:/Users/Slyexistence/AppData/Roaming/nltk_data')
nltk.download('stopwords', download_dir='C:/Users/Slyexistence/AppData/Roaming/nltk_data')

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize("Пример текста для проверки токенизации")
print(tokens)

['C:\\Users\\Slyexistence/nltk_data', 'c:\\Users\\Slyexistence\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data', 'c:\\Users\\Slyexistence\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data', 'c:\\Users\\Slyexistence\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data', 'C:\\Users\\Slyexistence\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']
['Пример', 'текста', 'для', 'проверки', 'токенизации']


[nltk_data] Downloading package punkt to
[nltk_data]     C:/Users/Slyexistence/AppData/Roaming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:/Users/Slyexistence/AppData/Roaming/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Часть А

In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import nltk


data = pd.read_csv("D:/ВУЗ/Атаева/Домашняя Работа/HomeWorkAnalyz/work_1/habr_articles.csv")
documents = data['Текст'].dropna().tolist()

nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
tokenizer = RegexpTokenizer(r'\w+')

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = tokenizer.tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

processed_docs = [preprocess_text(doc) for doc in documents]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Slyexistence\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF

vectorizer = CountVectorizer(max_df=0.9, min_df=10)
dtm = vectorizer.fit_transform(processed_docs)


lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)

print("Темы, найденные LDA:")
for idx, topic in enumerate(lda.components_):
    print(f"Тема {idx + 1}: {[vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]]}")

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10)
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_docs)

lsa = TruncatedSVD(n_components=5, random_state=42)
lsa.fit(tfidf_matrix)

print("\nТемы, найденные LSA:")
for idx, topic in enumerate(lsa.components_):
    print(f"Тема {idx + 1}: {[tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]]}")

nmf = NMF(n_components=5, random_state=42)
nmf.fit(tfidf_matrix)

print("\nТемы, найденные NMF:")
for idx, topic in enumerate(nmf.components_):
    print(f"Тема {idx + 1}: {[tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]]}")


Темы, найденные LDA:
Тема 1: ['linux', 'разработки', 'сегодня', 'разработчик', 'данных', 'таких', 'хабр', 'зовут', 'это', 'привет']
Тема 2: ['также', 'управления', 'разработки', 'системы', 'это', 'данной', 'недавно', 'эта', 'статья', 'статье']
Тема 3: ['python', 'назад', 'программирования', 'работы', 'статью', 'лет', 'пользователь', 'сделать', 'который', 'это']
Тема 4: ['безопасности', 'также', 'могут', 'очень', 'время', 'пользователей', 'поэтому', 'которые', 'данных', 'это']
Тема 5: ['зовут', 'поделиться', 'хочу', 'вами', 'расскажу', 'статье', 'всем', 'сегодня', 'хабр', 'привет']

Темы, найденные LSA:
Тема 1: ['данных', 'расскажу', 'компании', 'сегодня', 'статье', 'это', 'зовут', 'всем', 'хабр', 'привет']
Тема 2: ['сегодня', 'каждый', 'которая', 'который', 'которые', 'например', 'статье', 'данных', 'это', 'пользователь']
Тема 3: ['статья', 'лет', 'время', 'работы', 'также', 'который', 'которые', 'статье', 'данных', 'это']
Тема 4: ['которых', 'расскажу', 'помощью', 'часть', 'статье', '

c:\Users\Slyexistence\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


# B

In [ ]:
import spacy

nlp = spacy.load("ru_core_news_sm")

# Функция для выделения сущностей
def extract_entities(text):
    doc = nlp(text)
    persons = set()
    locations = set()
    organizations = set()
    
    for ent in doc.ents:
        if ent.label_ == "PER":  # Имена персон
            persons.add(ent.text)
        elif ent.label_ == "LOC":  # Географические названия
            locations.add(ent.text)
        elif ent.label_ == "ORG":  # Организации
            organizations.add(ent.text)
    
    return {
        "persons": persons,
        "locations": locations,
        "organizations": organizations
    }

entities_per_document = [extract_entities(doc) for doc in processed_docs]

all_persons = set()
all_locations = set()
all_organizations = set()

for entities in entities_per_document:
    all_persons.update(entities["persons"])
    all_locations.update(entities["locations"])
    all_organizations.update(entities["organizations"])

print("Упоминаемые персоны:", all_persons)
print("Географические названия:", all_locations)
print("Названия организаций:", all_organizations)


Упоминаемые персоны: {'татьяна ошуркова', 'игорь иванович', 'адаптирована ваши', 'константин', 'карл это', 'даша волкова', 'дмитрий рожков', 'дарья санькова', 'алексей зотов', 'екатерина шеленкова', 'александр aka bytehope', 'глеб михеев', 'александр михеев', 'александр карпов', 'романа сакутина', 'мария', 'антон комаров', 'алла работаю младшим', 'дмитрия медведева', 'юрием вашинко', 'джорджа буля', 'алана колмероэ', 'александра рябова', 'амир хотел', 'редактор хабра', 'даниил нейман', 'алексей карпенко', 'татьяна маркина', 'анжелика захарова', 'ефимов михаил', 'денис net backend', 'влюблён хираяма', 'дмитрий middlereactразработчик', 'бу', 'дмитрий фролов', 'артем мелехин', 'сергей щербаков', 'максим qa максилекте', 'сергей рыжков', 'кирилл', 'анастасия березовская', 'лекс фридман', 'александр патутинский', 'александр устюжанин', 'айо перевела', 'алексей жиряков', 'александр бобряков', 'дональд кнут мартин фаулер карлос буэно', 'филипенко', 'сергей рабинович', 'павел маслов', 'антон во